In [ ]:
## using csv files from 
# from https://covidclinicaldata.org/ 

In [ ]:
import pandas as pd
import glob

In [ ]:
# using approach from 
# https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe

In [ ]:
all_files = glob.glob('data/*.csv')
df = pd.concat((pd.read_csv(f) for f in all_files), sort=False)

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df.head(100)

In [ ]:
df.columns

In [ ]:
len(df)

In [ ]:
#df.fillna('NULL')

In [ ]:
df.describe()

In [ ]:
# map true and false to 1 and 0 (scikit-learn requires numerical input for classification)
#per: https://stackoverflow.com/questions/17383094/how-can-i-map-true-false-to-1-0-in-a-pandas-dataframe
df = df * 1

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

In [ ]:
pysqldf("SELECT MAX(batch_date), MIN(batch_date) FROM df ORDER BY batch_date DESC")

In [ ]:
pysqldf("SELECT covid19_test_results, count(covid19_test_results) as R FROM df GROUP BY covid19_test_results")

In [ ]:
df.columns

In [ ]:
s = "fatigue"

In [ ]:
sql = f"""
SELECT 
    {s} as S, covid19_test_results = 'Positive' as R
FROM df
WHERE {s} IS NOT NULL
"""

r = pysqldf(sql)

In [ ]:
r.describe()

In [ ]:
df.columns

In [ ]:
s = "loss_of_smell * loss_of_taste"

sql = f"""
SELECT 
    {s} as S, covid19_test_results = 'Positive' as R, count(*) as C
FROM df
WHERE {s} IS NOT NULL
GROUP BY {s}, covid19_test_results
"""

r = pysqldf(sql)

In [ ]:
r

In [ ]:
r.sum()['C']

In [ ]:
for i in range(1,len(r),4):
    print("Symptom Neg, Test Pos:", round(r.iloc[i]['C'] / (r.iloc[i-1]['C'] + r.iloc[i]['C']),3))
    print("Symptom Pos, Test Pos:", round(r.iloc[i+2]['C'] / (r.iloc[i+1]['C'] + r.iloc[i+2]['C']),3))

In [ ]:
pysqldf("""
SELECT COUNT(covid19_test_results) FROM
df
WHERE 
wheezes = 0
AND cough = 0
AND fever = 0 
AND sob = 0 
AND diarrhea = 0 
AND fatigue = 0 
AND headache = 0
AND loss_of_smell = 0 
AND loss_of_taste = 0 
AND runny_nose = 0 
AND  muscle_sore = 0 
AND sore_throat = 0
AND covid19_test_results = 'Positive'
""")